# Neural network classification function

If we want to do classification with neural networks we want to output a probability distribution over the classes from the output targets $t$. For the classification of 2 classes $t=1$ or $t=0$ we can use the [logistic function](http://en.wikipedia.org/wiki/Logistic_function) used in [logistic regression](http://en.wikipedia.org/wiki/Logistic_regression). For multiclass classification there exists an extention of this logistic function called the [softmax function](http://en.wikipedia.org/wiki/Softmax_function) which is used in [multinomial logistic regression](http://en.wikipedia.org/wiki/Multinomial_logistic_regression). The following sections will first explain the logistic function and how to optimise it, and hereafter it will explain the softmax function and how to derive it.

In [ ]:
# Python imports
import numpy as np # Matrix and vector computation package
import matplotlib.pyplot as plt  # Plotting library
from matplotlib.colors import colorConverter, ListedColormap # some plotting functions
from mpl_toolkits.mplot3d import Axes3D  # 3D plots
from matplotlib import cm # Colormaps
# Allow matplotlib to plot inside this notebook
%matplotlib inline

## Logistic function

The goal is to predict the target class $t$ from an input $z$. The probability $P(t=1 | z)$ that input $z$ is classified as class $t=1$ is represented by the output $y$ of the logistic function computed as $y = \sigma(z)$. $\sigma$ is the [logistic function](http://en.wikipedia.org/wiki/Logistic_function) and is defined as:
$$ \sigma(z) = \frac{1}{1+e^{-z}} $$

This logistic function maps the input $z$ to an output between $0$ and $1$ as is illustrated in the figure below.

We can write the probabilities that the class is $t=1$ or $t=0$ given input $z$ as:

$$ P(t=1| z) = \sigma(z) = \frac{1}{1+e^{-z}} $$
$$ P(t=0| z) = 1 - \sigma(z) = \frac{e^{-z}}{1+e^{-z}} $$

Note that the logistic function is derived from the log [odds ratio](http://www.ats.ucla.edu/stat/mult_pkg/faq/general/odds_ratio.htm) of $P(t=1|z)$ over $P(t=0|z)$.

$$\begin{split}
log \frac{P(t=1|z)}{P(t=0|z)} & = log \frac{\frac{1}{1+e^{-z}}}{\frac{e^{-z}}{1+e^{-z}}} = log \frac{1}{e^{-z}} \\
& = log(1) - log(e^{-z}) = z
\end{split}$$

This means that the logg odds ratio $log(P(t=1|z)/P(t=0|z))$ changes linearly with $z$. And if $z = x*w$ as in neural networks, this means that  the logg odds ratio changes linearly with the parameters $w$ and input samples $x$.

In [ ]:
# Define the logistic function
def logistic(z): return 1 / (1 + np.exp(-z))

# Plot the logistic function
z = np.linspace(-6,6,100)
plt.plot(z, logistic(z), 'b-')
plt.xlabel('$z$', fontsize=15)
plt.ylabel('$\sigma(z)$', fontsize=15)
plt.title('logistic function')
plt.grid()
plt.show()

### Derivative of the logistic function

Since neural networks typically use [gradient](http://en.wikipedia.org/wiki/Gradient) based opimization techniques such as [gradient descent](http://en.wikipedia.org/wiki/Gradient_descent) it is important to define the [derivative](http://en.wikipedia.org/wiki/Derivative) of the output $y$ of the logistic function with respect to its input $z$. ${\partial y}/{\partial z}$ can be calculated as:

$$\frac{\partial y}{\partial z} = \frac{\partial \sigma(z)}{\partial z} = \frac{\partial \frac{1}{1+e^{-z}}}{\partial z} = \frac{-1}{(1+e^{-z})^2} *e^{-z}*-1 = \frac{1}{1+e^{-z}} \frac{e^{-z}}{1+e^{-z}}$$

And since $1 - \sigma(z)) = 1 - {1}/(1+e^{-z}) = {e^{-z}}/(1+e^{-z})$ this can be rewritten as:

$$\frac{\partial y}{\partial z} = \frac{1}{1+e^{-z}} \frac{e^{-z}}{1+e^{-z}} = \sigma(z) * (1- \sigma(z)) =  y (1-y)$$

This derivative is plotted below.

In [ ]:
# Define the logistic function
def logistic_derivative(z): return logistic(z) * (1 - logistic(z))

# Plot the logistic function
z = np.linspace(-6,6,100)
plt.plot(z, logistic_derivative(z), 'r-')
plt.xlabel('$z$', fontsize=15)
plt.ylabel('$\\frac{\\partial \\sigma(z)}{\\partial z}$', fontsize=15)
plt.title('derivative of the logistic function')
plt.grid()
plt.show()

### Cross-entropy cost function for the logistic function

The output of the model $y = \sigma(z)$ can be interpreted as a probability $y$ that input $z$ belongs to one class $(t=1)$, or probability $1-y$ that $z$ belongs to the other class $(t=0)$ in a two class classification problem. We note this down as: $P(t=1| z) = \sigma(z) = y$.

The neural network model will be optimized by maximizing the [likelihood](http://en.wikipedia.org/wiki/Likelihood_function) that a given set of parameters $\theta$ of the model can result in prediction of the correct class of each input sample. The parameters $\theta$ transform each input sample $i$ into an input to the logistic function $z_{i}$. The likelihood maximization can be written as:

$$\underset{\theta}{\text{argmax}}\; \mathcal{L}(\theta|t,z) = \underset{\theta}{\text{argmax}} \prod_{i=1}^{n} \mathcal{L}(\theta|t_i,z_i)$$

The likelihood $\mathcal{L}(\theta|t,z)$ can be rewritten as the [joint probability](http://en.wikipedia.org/wiki/Joint_probability_distribution) of generating $t$ and $z$ given the parameters $\theta$: $P(t,z|\theta)$. Since $P(A,B) = P(A|B)*P(B)$ this can be written as:

$$P(t,z|\theta) = P(t|z;\theta)P(z|\theta)$$

Since we are not interested in the probability of $z$ we can reduce this to: $\mathcal{L}(\theta|t,z) = P(t|z;\theta) = \prod_{i=1}^{n} P(t_i|z_i;\theta)$. 
Since $t_i$ is a [Bernoulli variable](http://en.wikipedia.org/wiki/Bernoulli_distribution), and the probability $P(t| z) = y$ is fixed for a given $\theta$ we can rewrite this as: 

$$\begin{split}
P(t|z) & = \prod_{i=1}^{n} P(t_i=1|z_i)^{t_i} * (1 - P(t_i=1|z_i))^{1-t_i} \\
& = \prod_{i=1}^{n} y_i^{t_i} * (1 - y_i)^{1-t_i} \end{split}$$

Since the logartimic function is a monotone increasing function we can optimize the log-likelihood function $\underset{\theta}{\text{argmax}}\; log \mathcal{L}(\theta|t,z)$. This maximum will be the same as the maximum from the regular likelihood function. The log-likelihood function can be written as:

$$\begin{split} log \mathcal{L}(\theta|t,z) & = log \prod_{i=1}^{n} y_i^{t_i} * (1 - y_i)^{1-t_i} \\
& = \sum_{i=1}^{n} t_i log(y_i) + (1-t_i) log(1 - y_i)
\end{split}$$

Minimizing the negative of this function (minimizing the negative log likelihood) corresponds to maximizing the likelihood. This error function $\xi(t,y)$ is typically known as the [cross-entropy error function](http://en.wikipedia.org/wiki/Cross_entropy) (also known as log-loss):

$$\begin{split}
\xi(t,y) & = - log \mathcal{L}(\theta|t,z) \\
& = - \sum_{i=1}^{n} \left[ t_i log(y_i) + (1-t_i)log(1-y_i) \right] \\
& = - \sum_{i=1}^{n} \left[ t_i log(\sigma(z) + (1-t_i)log(1-\sigma(z) \right]
\end{split}$$

This function looks complicated but besides the previous derivation there are a couple of intuitions why this function is used as a cost function for logistic regression. First of all it can be rewritten as:

$$ \xi(t_i,y_i) = 
   \begin{cases}
   -log(y_i) & \text{if } t_i = 1 \\
   -log(1-y_i) & \text{if } t_i = 0
  \end{cases}$$
  
Which in the case of $t_i=1$ is $0$ if $y_i=1$ $(-log(1)=0)$ and goes to infinity as $y_i \rightarrow 0$ $(\underset{y \rightarrow 0}{\text{lim}}  -log(y) = +\infty)$. The reverse effect is happening if $t_i=0$.  
So what we end up with is a cost function that is $0$ if the probability to predict the correct class is $1$, and goes to infinity as the probability to predict the correct class goes to $0$.

Notice that the cost function $\xi(t,y)$ is equal to the negative [log probability](http://en.wikipedia.org/wiki/Log_probability) that $z$ is classified as its correct class:  
$-log(P(t=1| z)) = -log(y)$,  
$-log(P(t=0| z)) = -log(1-y)$.

By minimizing the negative log probability we will maximize the log probability. And since $t$ can only be $0$ or $1$, we can write $\xi(t,y)$ as:
$$ \xi(t,y) = -t log(y) - (1-t)log(1-y) $$

Which will give $\xi(t,y) = - \sum_{i=1}^{n} \left[ t_i log(y_i) + (1-t_i)log(1-y_i) \right]$ if we sum over all $n$ samples.



Another reason to use the cross-entropy function is that in simple logistic regression this results in a [convex](http://en.wikipedia.org/wiki/Convex_function) cost function, of which the global minimum will be easy to find. Note that this is not necessaraly the case anymore in multilayer neural networks.

## Softmax function


The [logistic output function](http://en.wikipedia.org/wiki/Logistic_function) described above can only be used for the classification between two target classes $t=1$ and $t=0$. This logistic function can be generalized to output a multiclass categorical probability distribution by the [softmax function](http://en.wikipedia.org/wiki/Softmax_function). This softmax function $\varsigma$ takes as input a $C$-dimensional vector $\mathbf{z}$ and outputs a $C$-dimensional vector $\mathbf{y}$ of real values between $0$ and $1$. This function is a normalized exponential and is defined as:

$$ y_c = \varsigma(\mathbf{z})_c = \frac{e^{z_c}}{\sum_{d=1}^C e^{z_d}} \quad \text{for} \; c = 1 \cdots C$$

The output $\sum_{d=1}^C e^{z_d}$ acts as a regularizer to make sure that $\sum_{c=1}^C y_c = 1$. 
As the output layer of a neural network the softmax function can be represented graphically as a layer with $C$ neurons.

We can write the probabilities that the class is $t=c$ for $c = 1 \cdots C$ given input $\mathbf{z}$ as:

$$ 
\begin{bmatrix} 
P(t=1 | \mathbf{z}) \\
\vdots \\
P(t=C | \mathbf{z}) \\
\end{bmatrix}
= 
\begin{bmatrix} 
\varsigma(\mathbf{z})_1 \\
\vdots \\
\varsigma(\mathbf{z})_C \\
\end{bmatrix}
= \frac{1}{\sum_{d=1}^C e^{z_d}}
\begin{bmatrix} 
e^{z_1} \\
\vdots \\
e^{z_C} \\
\end{bmatrix}
$$

Where $P(t=c | \mathbf{z})$ is thus the probability that that the class is $c$ given the input $\mathbf{z}$.

These probabilities for an example system with 2 classes ($t=1$, $t=2$) and input $\mathbf{z} = [z_1, z_2]$ is shown in the figure below. Notice that the outputs for both classes are complementary and sum to one.

In [ ]:
# Define the softmax function
def softmax(z): return np.exp(z) / np.sum(np.exp(z))

# Plot the softmax output for 2 dimensions for both classes
# Plot the output in function of the weights
# Define a vector of weights for which we want to plot the ooutput
nb_of_zs = 200
zs = np.linspace(-10, 10, num=nb_of_zs) # input 
zs_1, zs_2 = np.meshgrid(zs, zs) # generate grid
y = np.zeros((nb_of_zs, nb_of_zs, 2)) # initialize output
# Fill the output matrix for each combination of input z's
for i in xrange(nb_of_zs):
    for j in xrange(nb_of_zs):
        y[i,j,:] = softmax(np.asarray([zs_1[i,j], zs_2[i,j]]))
# Plot the cost function surfaces for both classes
fig = plt.figure(figsize=plt.figaspect(0.4))
# Plot the cost function surface for t=1
ax = fig.add_subplot(121, projection='3d')
surf1 = ax.plot_surface(zs_1, zs_2, y[:,:,0], linewidth=0, cmap=cm.coolwarm)
ax.view_init(elev=20, azim=60)
ax.set_xlabel('$z_1$', fontsize=15)
ax.set_ylabel('$z_2$', fontsize=15)
ax.set_zlabel('$y_1$', fontsize=15)
ax.set_title ('$P(t=1|\mathbf{z})$')
# Plot the cost function surface for t=1
ax = fig.add_subplot(122, projection='3d')
surf2 = ax.plot_surface(zs_1, zs_2, y[:,:,1], linewidth=0, cmap=cm.coolwarm)
ax.view_init(elev=20, azim=30)
ax.set_xlabel('$z_1$', fontsize=15)
ax.set_ylabel('$z_2$', fontsize=15)
ax.set_zlabel('$y_2$', fontsize=15)
ax.set_title ('$P(t=2|\mathbf{z})$')
fig.suptitle('Cost function surface for both classes')
plt.grid()
plt.show()

### Derivative of the softmax function

To use the softmax function in neural networks we need to compute its derivative. This derivative ${\partial y_i}/{\partial z_j}$ of the output $\mathbf{y}$ of the softmax function with respect to its input $\mathbf{z}$ can be calculated as:

$$
\begin{split}
\text{if} \; i = j :& \frac{\partial y_i}{\partial z_i} = \frac{\partial \frac{e^{z_i}}{e^{z_i}}}{\partial z_i} = \frac{e^{2z_i}-e^{2z_i}}{e^{2z_i}} = \frac{e^{z_i}}{e^{z_i}} \frac{e^{z_i}-e^{z_i}}{e^{z_i}} = \frac{e^{z_i}}{e^{z_i}} (1 - \frac{e^{z_i}}{e^{z_i}}) = y_i (1 - y_i) \\
\text{if} \; i \neq j :& \frac{\partial y_i}{\partial z_j} = \frac{\partial \frac{e^{z_i}}{e^{z_j}}}{\partial z_j} = \frac{0 - e^{z_i}e^{z_j}}{e^{2z_j}} = -\frac{e^{z_i}}{e^{z_j}} \frac{e^{z_j}}{e^{z_j}} = -y_i y_j
\end{split}$$

This can be written in one formula with the help of the [Kronecker delta](http://en.wikipedia.org/wiki/Kronecker_delta) $\delta_{ij}$ as:

$$ \frac{\partial y_i}{\partial z_j} = y_i (\delta_{ij} - y_j) $$

Note that if $i = j$ this derivative is similar to the derivative of the logistic function.


### Cross-entropy cost function for the softmax function